# XGBoost regression (single-node)

![xgboost](https://upload.wikimedia.org/wikipedia/commons/6/69/XGBoost_logo.png)

In [ ]:
import os
import warnings

import cloudpickle
import snowflake.connector
import xgboost

from sklearn.metrics import mean_squared_error

warnings.simplefilter("ignore")

In [ ]:
MODEL_PATH = 'models'
if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)
    
numeric_feat = [
    'pickup_weekday', 
    'pickup_weekofyear', 
    'pickup_hour', 
    'pickup_week_hour', 
    'pickup_minute', 
    'passenger_count',
]
categorical_feat = [
    'pickup_taxizone_id', 
    'dropoff_taxizone_id',
]
features = numeric_feat + categorical_feat
y_col = 'tip_fraction'

# Load data and feature engineering

Load a sample from a single month for this exercise

In [ ]:
SNOWFLAKE_ACCOUNT = os.environ['SNOWFLAKE_ACCOUNT']
SNOWFLAKE_USER = os.environ['SNOWFLAKE_USER']
SNOWFLAKE_PASSWORD = os.environ['SNOWFLAKE_PASSWORD']

SNOWFLAKE_WAREHOUSE = os.environ['SNOWFLAKE_WAREHOUSE']
TAXI_DATABASE = os.environ['TAXI_DATABASE']
TAXI_SCHEMA = os.environ['TAXI_SCHEMA']

conn_info = {
    'account': SNOWFLAKE_ACCOUNT,
    'user': SNOWFLAKE_USER,
    'password': SNOWFLAKE_PASSWORD,
    'warehouse': SNOWFLAKE_WAREHOUSE,
    'database': TAXI_DATABASE,
    'schema': TAXI_SCHEMA,
}
conn = snowflake.connector.connect(**conn_info)

In [ ]:
query = """
SELECT * FROM (
    SELECT 
        pickup_taxizone_id,
        dropoff_taxizone_id,
        passenger_count,
        DIV0(tip_amount, fare_amount) AS tip_fraction,
        DAYOFWEEKISO(pickup_datetime) - 1 AS pickup_weekday,
        WEEKOFYEAR(pickup_datetime) AS pickup_weekofyear,
        HOUR(pickup_datetime) AS pickup_hour,
        (pickup_weekday * 24) + pickup_hour AS pickup_week_hour,
        MINUTE(pickup_datetime) AS pickup_minute
    FROM taxi_yellow
    WHERE
        DATE_TRUNC('MONTH', pickup_datetime) = %s
) SAMPLE (30)
"""
taxi = conn.cursor().execute(query, '2019-01-01').fetch_pandas_all()
taxi.columns = taxi.columns.str.lower()

In [ ]:
print(f'Num rows: {len(taxi)}, Size: {taxi.memory_usage(deep=True).sum() / 1e6} MB')

In [ ]:
taxi_train = taxi[features + [y_col]].astype(float).fillna(-1)

In [ ]:
taxi_train.head()

# Train a model

Setting `nthread=-1` tells xgboost to use all available cores on this machine to parallelize model training

In [ ]:
xgb_reg = xgboost.XGBRegressor(
    objective="reg:squarederror",
    tree_method="approx",
    learning_rate=0.1,
    max_depth=5,
    n_estimators=50,
    n_jobs=4,
    verbosity=1
)

In [ ]:
%%time
_ = xgb_reg.fit(taxi_train[features], y=taxi_train[y_col])

## Save model

In [ ]:
with open(f'{MODEL_PATH}/xgboost.pkl', 'wb') as f:
    cloudpickle.dump(xgb_reg, f)

## Calculate metrics on test set

Use a different month for test set

In [ ]:
taxi_test = conn.cursor().execute(query, '2019-02-01').fetch_pandas_all()
taxi_test.columns = taxi_test.columns.str.lower()

In [ ]:
preds = xgb_reg.predict(taxi_test[features])
mean_squared_error(taxi_test[y_col], preds, squared=False)